<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ
</h1>

---

<h1 style="text-align: center;">
    Обучение нейрона с помощью функции потерь LogLoss
</h1>

## Напоминание

Почти любой алгоритм машинного обучения, решающий задачу **классификации** или **регрессии**, работает так:

1. **Стадия инициализации**: задаются его **гиперпараметры**, то есть те величины, которые не "выучиваются" алгоритмом в процессе обучения самостоятельно.

2. **Стадия обучения**: алгоритм запускается на данных, **обучаясь** на них и меняя свои **параметры** (не путать с *гипер*параметрами) каким-то определённым образом (например, с помощью *метода градиентного спуска* или *метода коррекции ошибки*), исходя из функции потерь (её называют *loss function*). Функция потерь, по сути, говорит, где и как ошибается модель.

3. **Стадия предсказания**: модель готова, и теперь с её помощью можно делать **предсказания** на новых объектах.

В данном ноутбуке будет решаться задача **бинарной классификации** с помощью нейрона:

* **Входные данные**: матрица $X$ размера $(n, m)$ и столбец $y$ из нулей и единиц размера $(n, 1)$. Строкам матрицы соответствуют объекты, столбцам - признаки (то есть строка $i$ есть набор признаков (**признаковое описание**) объекта $X_i$).

* **Выходные данные**: столбец $\hat{y}$ из нулей и единиц размера $(n, 1)$ - предсказания алгоритма.

## Нейрон с сигмоидой

Снова рассмотрим нейрон с сигмоидой, то есть:

$$
f(x) = \sigma(x)=\frac{1}{1+e^{-x}}
$$

На предыдущем занятии мы установили, что **обучение нейрона с сигмоидой с квадратичной функцией потерь**:  

$$
MSE(w, x) = \frac{1}{n}\sum_{i=1}^{n} (\hat{y_i} - y_i)^2 = \frac{1}{n}\sum_{i=1}^{n} (\sigma(w \cdot x_i) - y_i)^2
$$    

где $w \cdot x_i$ - скалярное произведение, а $\sigma(w \cdot x_i) =\frac{1}{1+e^{-w \cdot x_i}} $ - сигмоида, **неэффективно**, то есть мы увидели, что даже за большое количество итераций нейрон предсказывает плохо.

Давайте ещё раз взглянем на формулу для градиентного спуска от функции потерь $L$ по весам нейрона:

$$
\frac{\partial Loss}{\partial w} = \frac{2}{n} X^T (\sigma(w \cdot X) - y)\sigma(w \cdot X)(1 - \sigma(w \cdot X))
$$

А теперь смотрим на график сигмоиды:

<img src='../img/neuron_1.png' width=500>

**Её значения: числа от 0 до 1.**

Если получше проанализировать формулу, то теперь можно заметить, что, поскольку сигмоида принимает значения между $0$ и $1$ (а значит ($1-\sigma$) тоже принимает значения от $0$ до $1$), то мы умножаем $X^T$ на столбец $(\sigma(w \cdot X) - y)$ из чисел от $-1$ до $1$, а потом ещё на столбцы $\sigma(w \cdot X)$ и $(1 - \sigma(w \cdot X))$ из чисел от $0$ до $1$. Таким образом в лучшем случае $\frac{\partial{L}}{\partial{w}}$ будет столбцом из чисел, порядок которых максимум $0.01$ (в среднем, понятно, что если сигмоида выдаёт все $0$, то будет $0$, если все $1$, то тоже $0$). После этого мы умножаем на шаг градиентного спуска, который обычно порядка $0.001$ или $0.01$ максимум. То есть мы вычитаем из весов числа порядка $\sim 0.0001$. Медленновато спускаемся, не правда ли? Это называют **проблемой затухающих градиентов**.

Всё верно. В задачах классификации, в которых моделью является нейрон с сигмоидной функцией активации, предсказывающий "вероятности" принадлженостей к классам, почти никогда не используют квадратичную функцию потерь $L$. Вместо неё придумали другую прекрасную функцию потерь - **LogLoss**:

$$
LogLoss(\hat{y}, y) = -\frac{1}{n} \sum_{i=1}^n y_i \log(\hat{y_i}) + (1 - y_i) \log(1 - \hat{y_i}) = -\frac{1}{n} \sum_{i=1}^n y_i \log(\sigma(w \cdot x_i)) + (1 - y_i) \log(1 - \sigma(w \cdot x_i))
$$

где, как и прежде, $y$ - вектор из истинных значений классов, а $\hat{y}$ - вектор из предсказаний нейрона.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

%matplotlib inline

In [ ]:
def loss(y_pred, y):
    return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

Отметим, что сейчас речь идёт именно о **бинарной классификации** (на два класса), в многоклассовой классификации используется функция потерь под названием **кросс-энтропия**, которая является обобщением LogLoss'а на случай нескольких классов.

Почему же теперь всё будет лучше? Раньше была проблема умножения маленьких чисел в градиенте. Давайте посмотрим, что теперь:

* Для веса $w_j$:

  $$
  \frac{\partial Loss}{\partial w_j} = 
  -\frac{1}{n} \sum_{i=1}^n \left(\frac{y_i}{\sigma(w \cdot x_i)} - \frac{1 - y_i}{1 - \sigma(w \cdot x_i)}\right)(\sigma(w       \cdot x_i))_{w_j}' = -\frac{1}{n} \sum_{i=1}^n \left(\frac{y_i}{\sigma(w \cdot x_i)} - \frac{1 - y_i}{1 - \sigma(w \cdot       x_i)}\right)\sigma(w \cdot x_i)(1 - \sigma(w \cdot x_i))x_{ij} =
  $$
  
  $$
  -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{ij}
  $$
  
* Градиент $Loss$'а по вектору весов - это вектор, $j$-ая компонента которого равна $\frac{\partial Loss}{\partial w_j}$ (помним, что весов всего $m$):

  $$
  \begin{align}
      \frac{\partial Loss}{\partial w} &= \begin{bmatrix}
          -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{i1} \\
          -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{i2} \\
          \vdots \\
          -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{im}
      \end{bmatrix}
   \end{align}=\frac{1}{n} X^T \left(\hat{y} - y\right)
  $$

По аналогии с нейроном выведем формулу для свободного члена (bias'а) $b$:

$$
\frac{\partial Loss}{\partial b} = 
-\frac{1}{n} \sum_{i=1}^n \left(\frac{y_i}{\sigma(w \cdot x_i + b)} - \frac{1 - y_i}{1 - \sigma(w \cdot x_i + b)}\right)(\sigma(w \cdot x_i + b))_{b}' = -\frac{1}{n} \sum_{i=1}^n \left(\frac{y_i}{\sigma(w \cdot x_i + b)} - \frac{1 - y_i}{1 - \sigma(w \cdot x_i + b)}\right)\sigma(w \cdot x_i + b)(1 - \sigma(w \cdot x_i + b))*1 =
$$

$$
-\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)
$$

Получили новое правило для обновления. Заметим, что это в точности то правило обновления весов для градиентного спуска, которое мы использовали для перцептрона (только другая функция активации). Получается, что мы пришли к этому правилу "по-честному", сделав функцией активации сигмоиду и введя новую функию потерь - LogLoss, а когда реализовывали перцептрон, мы просто сказали (воспользовавшись **правилом Хебба**), что $f'(x)$ возьмём единицей, то есть тут имеет место интересная связь градиентного спуска и метода коррекции ошибки.

Отсюда очевидно, что код для нейрона с такой функцией потерь не будет ничем отличаться от кода для перцептрона, за исключением метода `self.activate` и самого подсчёта `loss`. Напишем:

In [ ]:
def sigmoid(x):
    """
    Сигмоидная функция
    """
    
    return 1 / (1 + np.exp(-x))

Реализуем нейрон с функцией потерь LogLoss:

In [ ]:
class Neuron:      
    def __init__(self, w=None, b=0):
        """
        param w - вектор весов
        param b - смещение
        """

        # пока что мы не знаем размер матрицы X, а значит не знаем, сколько будет весов
        self.w = w
        self.b = b
        
    def activate(self, x):
        return sigmoid(x)
        
    def forward_pass(self, X):
        """
        Рассчитывает ответ нейрона при предъявлении набора объектов
            param X - матрица объекты-признаки размера (n, m), каждая строка - отдельный объект
            return - вектор размера (n,) из нулей и единиц с ответами нейрона
        """
        
        y_pred = self.activate(X @ self.w + self.b)
        return y_pred
    
    def backward_pass(self, X, y, y_pred, lr=0.1):
        """
        Обновляет значения весов нейрона
            param X - матрица входов размера (n, m)
            param y - вектор правильных ответов размера (n,)
            param lr - "скорость обучения" (символ alpha в формулах выше)
        
        В этом методе ничего возвращать не нужно, только правильно
        поменять веса с помощью градиентного спуска.
        """
        
        n = X.shape[0]
        
        # осторожнее с порядком операций
        self.w -= lr / n * X.T @ (y_pred - y)
        self.b -= lr * np.mean(y_pred - y)

    def fit(self, X, y, num_epochs=5000):
        """
        Спускаемся в минимум
            param X - матрица объектов размера (n, m)
            param y - вектор правильных ответов размера (n,)
            param num_epochs - количество итераций обучения
            return - вектор значений функции потерь
        """
        
        # вектор размера (m,)
        self.w = np.zeros(X.shape[1])
        # смещение
        self.b = 0
        # значения функции потерь на различных итерациях обновления весов
        losses = []
        
        for i in range(num_epochs):
            # предсказания с текущими весами
            y_pred = self.forward_pass(X)
            # считаем функцию потерь с текущими весами
            losses.append(loss(y_pred, y))
            # обновляем веса в соответсвии с тем, где ошиблись раньше
            self.backward_pass(X, y, y_pred)
        
        return losses

## Тестирование нейрона

Протестируем нейрон, обученный с новой функцией потерь, на тех же данных, что и в предыдущем ноутбуке:

#### Проверка `forward_pass`

In [ ]:
w = np.array([1., 2.])
b = 2.
X = np.array([[1., 3.],
              [2., 4.],
              [-1., -3.2]])

neuron = Neuron(w, b)
y_pred = neuron.forward_pass(X)
print(y_pred)

#### Проверка `backward_pass`

In [ ]:
y = np.array([1, 0, 1])

In [ ]:
neuron.backward_pass(X, y, y_pred)

print(neuron.w)
print(neuron.b)

Загрузим данные (яблоки и груши):

In [ ]:
data = pd.read_csv('data/apples_pears.csv')

In [ ]:
data.head()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(data.iloc[:, 0], data.iloc[:, 1], c=data['target'], cmap='rainbow')
plt.title('Яблоки и груши', fontsize=15)
plt.xlabel('Симметричность', fontsize=14)
plt.ylabel('Желтизна', fontsize=14)
plt.show()

Обозначим, что здесь признаки, а что - классы:

In [ ]:
# матрица объекты-признаки
X = data.iloc[:, :2].values
# классы (столбец из нулей и единиц)
y = data['target'].values

**Вывод функции потерь.** Функция потерь должна убывать и в итоге стать близкой к $0$.

In [ ]:
# может вызывать баги на старых версиях jupyter notebook
# %%time

neuron = Neuron()
losses = neuron.fit(X, y)

plt.figure(figsize=(10, 8))
plt.plot(losses)
plt.title('Функция потерь', fontsize=15)
plt.xlabel('Номер итерации', fontsize=14)
plt.ylabel('$Loss(\hat{y}, y)$', fontsize=14)
plt.show()

Посмотрим, как нейрон классифицировал объекты из выборки:

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(data.iloc[:, 0], data.iloc[:, 1], c=np.array(neuron.forward_pass(X) > 0.5).ravel(), cmap='spring')
plt.title('Яблоки и груши', fontsize=15)
plt.xlabel('Симметричность', fontsize=14)
plt.ylabel('Желтизна', fontsize=14)
plt.show()

Видим, что с **LogLoss** в случае классификации работать лучше, чем с квадратичной функцией потерь (с $5000$ итераций и `lr=0.1` здесь лучше, чем с $5000$ итераций и `lr=0.1` в нейроне с **MSE**). 

## Полезные ссылки

1. Статья от Стэнфорда - http://cs231n.github.io/neural-networks-1/

2. Хорошая статья про функции активации - https://www.jeremyjordan.me/neural-networks-activation-functions/

3. Видео от Siraj Raval - https://www.youtube.com/watch?v=-7scQpJT7uo

4. Современная статья про функции активации. Теперь на хайпе активация $swish(x) = x\sigma (\beta x)$: https://arxiv.org/pdf/1710.05941.pdf (кстати, при её поиске в некоторой степени использовался neural architecture search)

5. **SeLU** - имеет очень интересные, доказанные с помощью теории вероятностей свойства: https://arxiv.org/pdf/1706.02515.pdf (да, в этой статье 102 страницы)